In [1]:
# import os
# os.environ['OPENAI_API_KEY'] = "..."

from dotenv import load_dotenv
load_dotenv()

True

## Load Document

In [2]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs = loader.load()

In [ ]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("../../modules/state_of_the_union.txt")
documents = loader.load()

## Process Document

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)

## Vector Store

### FAISS

In [4]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
vector = FAISS.from_documents(documents, embeddings)

print(documents[0])

page_content='LangSmith User Guide | 🦜️🛠️ LangSmith' metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': 'LangSmith User Guide | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.', 'language': 'en'}


### Chroma

In [7]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# load it into Chroma
vector = Chroma.from_documents(docs, embedding_function)

## Prompt

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

## LLM

In [9]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()


## Chain

In [10]:
document_chain = create_stuff_documents_chain(llm, prompt)

## Usage

In [11]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [12]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])


Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


LangSmith can help with testing by allowing developers to create datasets, run tests on LLM applications, upload test cases in bulk, create custom evaluations, compare results for different configurations, use a playground environment for rapid iteration and experimentation, collect feedback from users, annotate traces, add runs to datasets, and monitor key metrics over time.
